In [0]:
df = spark.sql(f"SELECT * FROM dev.job_prospects.job_1900_silver")
df.display()


In [0]:
area_list = [
    "Cao Bằng",
    "Sơn La",
    "Lai Châu",
    "Lạng Sơn",
    "Tuyên Quang",
    "Lào Cai",
    "Thái Nguyên",
    "Điện Biên",
    "Phú Thọ",
    "Bắc Ninh",
    "Hà Nội",
    "Quảng Ninh",
    "Hải Phòng",
    "Hưng Yên",
    "Ninh Bình",
    "Thanh Hóa",
    "Nghệ An",
    "Hà Tĩnh",
    "Quảng Trị",
    "Huế",
    "Đà Nẵng",
    "Quảng Ngãi",
    "Gia Lai",
    "Đắk Lắk",
    "Khánh Hòa",
    "Lâm Đồng",
    "Đồng Nai",
    "Thành phố Hồ Chí Minh",
    "Tây Ninh",
    "Đồng Tháp",
    "Vĩnh Long",
    "Cần Thơ",
    "An Giang",
    "Cà Mau"
]

In [0]:
# The error is due to using array<string> directly in from_json. In Databricks SQL, use 'ARRAY<STRING>' instead.

import json

df.createOrReplaceTempView("df_view")

area_list_str = json.dumps(area_list)

classified_area_df = spark.sql(
    f"""
    SELECT *,
        from_json(
            ai_query(
                'databricks-meta-llama-3-3-70b-instruct',
                'Given the following area names as string separated by "," or "|", classify each into one or more of the following valid areas:\n {area_list_str}\nReturn a list of matched area(s) for each input in only an JSON array format, no comments whatsoever. Input:\n' || area || "For example, if the input is 'Hà Nội,quận 1, TP. HCM|Hải Phòng', the output should be a JSON format: ['Hà Nội', 'TP. HCM', 'Hải Phòng']"
            ),
            'ARRAY<STRING>'
        ) AS area_classified
    FROM df_view
    """
)
# display(classified_area_df)

In [0]:
inferred_area_df = classified_area_df.withColumn(
    "area", classified_area_df["area_classified"]
).drop("area_classified")
# inferred_area_df.display()

In [0]:
inferred_area_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").option("batchSize", 10000).saveAsTable("dev.job_prospects.job_1900_silver_inferred")